<a target="_blank" href="https://colab.research.google.com/github/modelica/fmi-beginners-tutorial-2023/tree/main/part3/tutorial_multiple_FMUs/interaction_with_ multiple_fmus.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Running Co-simulations with the INTO-CPS Maestro Co-simulation Engine

The goal of this notebook is to highlight how to run a co-simulation using an open source tool.

It is meant to represent one of the simplest ways to get started running co-simulations with a command line tool.

The notebook is organized into two parts:
1. The first part is focused on running a co-simulation from the command line without concern for the accuracy of the results.
2. The second part shows how the accuracy of the co-simulation can be measured provided that there exists a baseline reference.

Running a co-simulation using the INTO-CPS co-simulation orchestration requires the user to specify the co-simulation scenario in a JSON file, which is then fed into the tool which will load the FMUS and produce the results as a CSV file.

In [ ]:
# Import all dependencies
import json
import subprocess
import pandas
import matplotlib.pyplot as plt
import numpy as np
import time


In [ ]:
%%bash
git clone https://github.com/modelica/fmi-beginners-tutorial-2023.git

In [ ]:
# Small details: make the jar executable in your platform.
%cd fmi-beginners-tutorial-2023/part3/tutorial_multiple_FMUs
coe_jar = "coe-1.0.10-jar-with-dependencies.jar"


# Make jar executable.
command = f"chmod +x {coe_jar}"
result = subprocess.run(command, shell=True, capture_output=True, text=True)
if result.returncode != 0:
    raise RuntimeError(f"Error running command {command}: {result.stderr}")


## Part 1: Running Co-simulations from the Command Line

This part is divided into 2 sections:
1. Running a co-simulation of a single FMU.
2. Running a co-simulation with multiple interconnected FMUs.

### Running Co-simulations with Single FMU

In [ ]:
# Prepare the co-simulation scenario description, which points to where the FMU.
singlefmu_scenario_json_file = "single_fmu_scenario.json"

# The following is a small detail whereby we need to find the absolute path of the FMU that is compatible with the platform of the user.
platform_folder = 'Linux'
# We can now see the co-simulation scenario as a json file:
with open(singlefmu_scenario_json_file, 'r') as file :
    scenario_raw = file.read()

print(json.dumps(json.loads(scenario_raw), indent=2))

In [ ]:
# Now we can run the co-simulation
coe_command = f"java -jar {coe_jar} -o -c {singlefmu_scenario_json_file} --starttime 0 --endtime 1"
result = subprocess.run(coe_command, shell=True, capture_output=True, text=True)

# Check the command's output
if result.returncode == 0:
    print(result.stdout)
else:
    raise RuntimeError(f"Error running command {coe_command}: {result.stderr}")

In [ ]:
# We can see the results stored in "output.csv"
data = pandas.read_csv("output.csv")
data

In [ ]:
# As can can plot those results as follows
plt.figure()
plt.plot(data["time"], data["{baseline}.baselineInstance.w"], label="{baseline}.w")
plt.legend()
plt.show()

### Running a Co-simulation with Multiple FMUs

In [ ]:
# Prepare the co-simulation scenario description, which points to where the FMUs are.

decoupled_scenario_json_file = "decoupled_scenario.json"

# We can now see the co-simulation scenario as a json file:
with open(decoupled_scenario_json_file, 'r') as file :
    scenario_raw = file.read()

print(json.dumps(json.loads(scenario_raw), indent=2))

In [ ]:
# Now we can run the co-simulation
coe_command = f"java -jar {coe_jar} -o -c {decoupled_scenario_json_file} --starttime 0 --endtime 1"
result = subprocess.run(coe_command, shell=True, capture_output=True, text=True)

# Check the command's output
if result.returncode == 0:
    print(result.stdout)
else:
    raise RuntimeError(f"Error running command {coe_command}: {result.stderr}")

In [ ]:
# If all went well, we can see the results stored in "output.csv"
data = pandas.read_csv("output.csv")
data

In [ ]:
# As can can plot those results as follows
plt.figure()
plt.plot(data["time"], data["{drive}.driveInstance.w"], label="{drive}.w")
plt.plot(data["time"], data["{stimuli}.stimuliInstance.w_desired"], label="{stimuli}.w_desired")
plt.legend()
plt.show()

## Part 2: Measuring Accuracy of the Co-simulation Wrt to Baseline

In this part we show how to measure the accuracy of a co-simulation and the impact of the co-simulation step size in the accuracy.
We have two sections:
1. We run a co-simulation scenario where we include the baseline as an FMU that is run in parallel with the other fmus, that are representing the same system, but in a decoupled manner.
2. In the second section we run multiple experiments with different communication step si in in order to highlight the impact of this parameter in the accuracy.

### Co-simulation Scenario with Baseline

In [ ]:
# Prepare the co-simulation scenario description, which points to where the FMUs are.
baseline_scenario_json_template = "baseline_scenario_template.json"
baseline_scenario_json_file = "baseline_scenario.json"

# We can now see the co-simulation scenario as a json file:
with open(baseline_scenario_json_file, 'r') as file:
    scenario_raw = file.read()

print(json.dumps(json.loads(scenario_raw), indent=2))

In [ ]:
# Now we can run the co-simulation
coe_command = f"java -jar {coe_jar} -o -c {baseline_scenario_json_file} --starttime 0 --endtime 1"
result = subprocess.run(coe_command, shell=True, capture_output=True, text=True)

# Check the command's output
if result.returncode == 0:
    print(result.stdout)
else:
    raise RuntimeError(f"Error running command {coe_command}: {result.stderr}")

In [ ]:
# If all went well, we can see the results stored in "output.csv"
data = pandas.read_csv("output.csv")
data

In [ ]:
# And we can plot those results as follows

plt.figure()
plt.plot(data["time"], data["{baseline}.baselineInstance.w"], label="{baseline}.w")
plt.plot(data["time"], data["{drive}.driveInstance.w"], label="{drive}.w")
plt.plot(data["time"], data["{stimuli}.stimuliInstance.w_desired"], label="{stimuli}.w_desired")
plt.legend()
plt.show()

In [ ]:
# Now we measure the error and print it:

def error(ref_series, approx_series):
    error_signal = ref_series.to_numpy() - approx_series.to_numpy()
    sum_squared_differences = np.sum(error_signal ** 2)
    return sum_squared_differences

error(data["{baseline}.baselineInstance.w"], data["{drive}.driveInstance.w"])

### Impact of Step Size in the Accuracy

This section is a bit more advanced.
What we want is to run many simulations with different step sizes, and for each of these runs, we want to collect the results, calculate their error and store it.
Then We can create a plot that compares the different step sizes with the achieved error.

In [ ]:
# So we need a function to run a full experiment given a step size.
def searchAndReplace(key, value, fileSrc, fileTrg):
  '''
  Opens a file fileSrc and searches for all occurrences of key string and replaces it by value.
  Stores the result in fileTrg.
  '''
  with open(fileSrc, 'r') as file :
    filedata = file.read()

  filedata = filedata.replace(key, value)

  with open(fileTrg, 'w') as file:
    file.write(filedata)

def run_experiment_with_step_size(step_size):
  # Set the step size in the config file:
  searchAndReplace("0.001", str(step_size), baseline_scenario_json_template, baseline_scenario_json_file)

  start_time_seconds = time.time()
  result = subprocess.run(coe_command, shell=True, capture_output=True, text=True)
  elapsed_time_seconds = time.time() - start_time_seconds

  # Check for failure and print debug information
  if result.returncode != 0:
    with open(baseline_scenario_json_file, 'r') as file:
        scenario_raw = file.read()
    raise RuntimeError(f"Error running command {coe_command} for step size {step_size}: {result.stderr}. \nThe input scenario is:\n{json.dumps(json.loads(scenario_raw), indent=2)}")

  # Read results and compute error
  data = pandas.read_csv("output.csv")

  return error(data["{baseline}.baselineInstance.w"], data["{drive}.driveInstance.w"]), elapsed_time_seconds

# Define possible step sizes we wish to try.
step_sizes = np.linspace(0.0001, 0.001, 4)
print(f"step_sizes = {step_sizes}")

# Run an experiment for each step size. CAUTION: this takes time.
results = [run_experiment_with_step_size(step_size) for step_size in step_sizes]
errors = [r[0] for r in results]
execution_times = [r[1] for r in results]

print(f"errors = {errors}")
print(f"execution_times = {execution_times}")

In [ ]:
# Plot the results

fig, axs = plt.subplots(2, 1)

axs[0].plot(step_sizes, errors, label="Error")
axs[0].legend()

axs[1].plot(step_sizes, execution_times, label="Execution time")
axs[1].legend()

# Adjust spacing between subplots
plt.tight_layout()